# Average of the monthly nighttime-lights

Nighttime-lights are available on the [server](https://www.ngdc.noaa.gov/eog/viirs/download_dnb_composites.html) as monthly products. For the study of detecting new built-up areas night lights of 2017 is used. Hence, the monthly products of nighttime-lights for the year of 2017 should be averaged.

In [3]:
import os
import glob

import numpy as np
import rasterio

import matplotlib.pylab as plt

#RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
import warnings
warnings.filterwarnings("always")

In [4]:
%matplotlib inline

In [67]:
DATA_PATH = '/media/io/Data/ASTROSAT_projects/Nighttime_lights/Monthly_night_lights'
nightlight_data_path = sorted(glob.glob(os.path.join(DATA_PATH, '2018','*.tif')))

### Average of monghtly night-lights.

In [22]:
nightlights_annual = {}

for path in nightlight_data_path:
    SVDNB_name = os.path.basename(path)
    SVDNB_name = os.path.splitext(SVDNB_name[:35])[0]
    with rasterio.open(path, 'r') as src:
        band = src.read(1)
        band_crs = src.crs
        transform = src.transform
        nightlights_annual[SVDNB_name] = {'band':band,
                                          'transform':transform,
                                          'crs':band_crs}
        
total = 0
for k,v in nightlights_annual.items():
    total += v['band']
    
nightlights_average = total / len(nightlights_annual)

### Save the raster file

In [72]:
profile = src.profile
profile.update(driver='GTiff', 
               dtype=rasterio.float32,
               compress='lzw',
               count=1)

with rasterio.open('/home/io/Desktop/annual_nightlights_2018.tif', 'w', **profile) as dst:
    dst.write(nightlights_average.astype(rasterio.float32), 1)